In [24]:
import polars as pl
import io
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder

# Scuccorese food ingredients data set
df = pl.read_parquet('hf://datasets/Scuccorese/food-ingredients-dataset/data/train-*.parquet')

In [25]:
# Removing uneccessary data
df = df.drop('ingredient', 'subcategory')
df = df.unnest("image").select(pl.col("category"), pl.col("bytes").alias("image"))

In [26]:
# Function to convert images to WEBP format
def convert_to_webp(image_bytes):
    with Image.open(io.BytesIO(image_bytes)) as im:
        if im.mode == "P":
            im = im.convert("RGBA")
        elif im.mode != "RGB":
            im = im.convert("RGB")

        # Save the image to WEBP format in memory
        output = io.BytesIO()
        im.save(output, format='WEBP')
        return output.getvalue()

# Convert all images to WEBP format
df = df.with_columns(
    pl.col("image").map_elements(convert_to_webp, return_dtype=pl.Binary)
)

df.head()

category,image
str,binary
"""vegetables""","b""RIFF\xb4\xed\x01\x00WEBPVP8\x20\xa8\xed\x01\x00\xf0\x09\x0b\x9d\x01*+\x04\x9a\x06>m4\x96G$#,\xab\xa7\xf1\xaa\xe1\x90\x0d\x89M\xdf{\x04\xb8\xfe\x1f\x03\x17\x9aq\x06\x9b!""..."
"""vegetables""","b""RIFF\xb2\x91\x01\x00WEBPVP8\x20\xa6\x91\x01\x00\xd0s\x04\x9d\x01*\x16\x02\x20\x03>m,\x92E\xa4""\xa1\x97I\xc7,@\x06\xc4\xb25\xb67\x8b\xa9\xb0#\xab07\x806G\x00""..."
"""vegetables""","b""RIFF\x0e\xc9\x05\x00WEBPVP8\x20\x02\xc9\x05\x00\x10[\x14\x9d\x01*\xb0\x04\x08\x07>m.\x93F\xa4""\xa6\xa9\xab\x13|\xf10\x0d\x89bl\xf2\x95W\xdf\x0c\xc4\x87\xe3\xfa\x16\xff?""..."
"""vegetables""","b""RIFF\xc8\xaa\x01\x00WEBPVP8\x20\xbc\xaa\x01\x00\x10\x89\x08\x9d\x01*\x14\x05\xbc\x03>m0\x95H$""\xa9\xad\xa5\x90k\xb9\xb0\x0d\x89ens\xa8\xb9%\xee:%\x8e&\x93\x93\xc5""..."
"""vegetables""","b""RIFFn\x8b\x02\x00WEBPVP8\x20b\x8b\x02\x00p\xb7\x13\x9d\x01*I\x08\x86\x05>m2\x95H$""\xb0\xad\xa5\xf1\xba\xea\x10\x0d\x89el\xcbN\xa1\x7f\xfd\xf3q\xe7\x9d\xea\xbf\xed""..."


In [27]:
# Function to decode webp into nparrays.
def decode_image(image_bytes):
    with Image.open(io.BytesIO(image_bytes)) as img:
        img = img.convert('RGB')
        img = img.resize((128, 128))
        img_array = np.array(img, dtype=np.float32) / 255.0
        return img_array


images = [decode_image(img_bytes) for img_bytes in df["image"].to_list()]
labels = df["category"].to_list()

In [28]:
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

one_hot_encoder = OneHotEncoder(sparse_output=False)
one_hot_labels = one_hot_encoder.fit_transform(encoded_labels.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(images, one_hot_labels, test_size=0.2)

X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (5340, 128, 128, 3)
y_train shape: (5340, 12)


In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [30]:
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.models import Sequential

# Define the CNN model
model = tf.keras.Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))


model.add(Dense(12, activation='softmax'))

model.summary()



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 128, 128, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 128, 128, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 128, 128, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 64, 64, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 64, 64, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ (None, 64, 64, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ (None, 32, 32, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 32, 32, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_19               │ (None, 32, 32, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 17,074,732 (65.13 MB)

 Trainable params: 17,072,812 (65.13 MB)

 Non-trainable params: 1,920 (7.50 KB)

In [31]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [32]:
# Train the model
history = model.fit(train_dataset, validation_data=test_dataset, epochs=10, callbacks=[lr_scheduler])

Epoch 1/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 963s 6s/step - accuracy: 0.2347 - loss: 2.6388 - val_accuracy: 0.1332 - val_loss: 3.8239 - learning_rate: 0.0010
Epoch 2/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 972s 6s/step - accuracy: 0.3571 - loss: 2.0741 - val_accuracy: 0.1751 - val_loss: 2.5549 - learning_rate: 0.0010
Epoch 3/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 950s 6s/step - accuracy: 0.4154 - loss: 1.7805 - val_accuracy: 0.2889 - val_loss: 2.4275 - learning_rate: 0.0010
Epoch 4/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 934s 6s/step - accuracy: 0.4629 - loss: 1.6402 - val_accuracy: 0.3817 - val_loss: 1.9260 - learning_rate: 0.0010
Epoch 5/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 973s 6s/step - accuracy: 0.5140 - loss: 1.4020 - val_accuracy: 0.4012 - val_loss: 1.9053 - learning_rate: 0.0010
Epoch 6/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 942s 6s/step - accuracy: 0.5295 - loss: 1.3551 - val_accuracy: 0.3997 - val_loss: 1.9372 - learning_rate: 0.0010
Epoch 7/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 968s 5s/step - accuracy: 0.6011 - loss: 1.

In [33]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

42/42 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.4476 - loss: 1.7880
Test Loss: 1.7991409301757812
Test Accuracy: 0.44610777497291565
